Lyrics Generator

In [1]:
pip install transformers==4.28.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.0 MB/s eta 0:00:00


In [ ]:
pip install --upgrade torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━ 397.8/619.9 MB 87.5 MB/s eta 0:00:03
ERROR: Operation cancelled by user


In [2]:
pip install pronouncing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 17.3 MB/s eta 0:00:00
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6237 sha256=f2db3decf3c4f3e87ac4a0f96c622eae57444bb464cb900cdc04f00c9756c51b
  Stored in directory: /root/.cache/pip/wheels/05/f6/1d/599c67da1fa48c086d8c49e8fc6bd5f05bc9fa66fb04bed5db
Successfully built pronouncing


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.4 MB/s eta 0:00:00


In [4]:
import random
import pronouncing
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
def extract_rhymes(text):
    lines = text.split('\n')
    rhyming_pairs = []

    for i in range(len(lines) - 1):
        line1 = lines[i].split()
        line2 = lines[i + 1].split()

        if line1 and line2:
            last_word1 = line1[-1]
            last_word2 = line2[-1]
            if last_word1 in pronouncing.rhymes(last_word2):
                rhyming_pairs.append(lines[i])
                rhyming_pairs.append(lines[i + 1])

    return '\n'.join(rhyming_pairs)

# Prepare the training data
train_file = 'Eminem.txt'
with open(train_file, 'r') as f:
    text = f.read()

rhyming_text = extract_rhymes(text)

with open('rhyming_emiem.txt', 'w') as f:
    f.write(rhyming_text)

class GPT2RhymeModel(GPT2LMHeadModel):
    def __init__(self, config, tokenizer):
        super().__init__(config)
        self.tokenizer = tokenizer
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        outputs = super().forward(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        lm_logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss()

        # Compute standard language modeling loss
        lm_loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))

        # Add custom rhyme loss
        rhyme_loss = compute_rhyme_loss(lm_logits, labels, self.tokenizer)
        total_loss = 0.9*lm_loss + 0.1*rhyme_loss.item()
        return (total_loss, outputs)

def compute_rhyme_loss(lm_logits, labels, tokenizer):
    # Convert logits to word tokens
    pred_ids = torch.argmax(lm_logits, dim=-1)
    pred_tokens = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    target_tokens = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rhyme_loss = 0.0
    rhyme_count = 0

    for batch_idx in range(len(pred_tokens)):
        pred_word = pred_tokens[batch_idx]
        target_word = target_tokens[batch_idx]

        if target_word in pronouncing.rhymes(pred_word):
            rhyme_count += 1

    # Calculate the rhyme percentage and convert it to a PyTorch tensor
    rhyme_percentage = 1.0 - (rhyme_count / len(pred_tokens))
    rhyme_loss = torch.tensor(rhyme_percentage, dtype=torch.float32)

    return rhyme_loss
    

In [ ]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

train_dataset = TextDataset(tokenizer=tokenizer, file_path='rhyming_emiem.txt', block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    save_steps=1000,
    logging_steps=1000,
    logging_dir='./logs',
    prediction_loss_only=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=200, training_loss=1.9375967407226562, metrics={'train_runtime': 4351.9953, 'train_samples_per_second': 0.655, 'train_steps_per_second': 0.046, 'total_flos': 186170572800000.0, 'train_loss': 1.9375967407226562, 'epoch': 50.0})

In [6]:
import pickle

In [ ]:
import pickle
# Save the model to a file
with open('EminemBetamodel', 'wb') as file:
    pickle.dump(model, file)

In [49]:
#Load the model from the file
with open('EminemBetamodel', 'rb') as file:
    model = pickle.load(file) 

In [10]:
# Load the tokenizer 
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [58]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

#load_directory = "trained_model"
#tokenizer = GPT2Tokenizer.from_pretrained(load_directory)
#model = GPT2RhymeModel.from_pretrained(load_directory, tokenizer=tokenizer)

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

prompt = "Fall to the ground and I wake up"
generated_lyrics = generator(prompt, max_length=120, num_return_sequences=1, temperature=0.8)

print(generated_lyrics[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fall to the ground and I wake up in your sleep
And it's like watching Casey Anthony walk in your neighborhood
Thoughts are falling all around you like they were shot
But you're not falling from that tree, you're in flight mode
And I'm standing in your path, your mind is blank
And I'm screaming in your earlobbery like it's my only chance to shut you down
‘Cause you're not even on your radar anymore
And if you think I'm insane, just sit your car on the street
And I'd rather kill you than


Post-trained Filter

In [13]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove.6B.300d.txt'  # Path to the downloaded GloVe file
word2vec_output_file = 'glove.6B.300d.word2vec.txt'

glove2word2vec(glove_input_file, word2vec_output_file)

<ipython-input-14-a06efa3873ff>:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 300)

In [16]:
from gensim.models import KeyedVectors

filename = 'glove.6B.300d.word2vec.txt'
mod = KeyedVectors.load_word2vec_format(filename, binary=False)


In [17]:
import string

def remove_punctuation(s):
    return ''.join(c for c in s if c not in string.punctuation)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
generated=generated_lyrics[0]['generated_text']
lines = generated.splitlines()
for i in range(len(lines)//2-1):
  first=lines[0].split()
  first_last=first[-1]
  first_last = remove_punctuation(first_last)
  rhymes = pronouncing.rhymes(first_last)
  line=lines[i+1].split()
  last=line[-1]
  last= remove_punctuation(last)
  sim=[0]*len(rhymes)
  try:
    vector1 = mod[last]
    for j in range(len(rhymes)):
      try:
        vector2= mod[rhymes[j]]
        vector1 = vector1.reshape(1, -1)
        vector2 = vector2.reshape(1, -1)  
        similarity = cosine_similarity(vector1, vector2)
        sim[j]=similarity
      except:
        continue
    sim=pd.DataFrame(sim)
    rhymes=pd.DataFrame(rhymes)
    sim.insert(loc=1,column=1,value=rhymes)
    sim=sim.sort_values(by=0)
    rhymes=list(sim[1])
    line[-1]=rhymes[-3]
    separator = ' '
    lines[i+1]=separator.join(line)
  except:
      continue

for i in range(len(lines)//2,len(lines)-1):
  first=lines[len(lines)//2].split()
  first_last=first[-1]
  first_last = remove_punctuation(first_last)
  rhymes = pronouncing.rhymes(first_last)
  line=lines[i+1].split()
  last=line[-1]
  last= remove_punctuation(last)
  sim=[0]*len(rhymes)
  try:
    vector1 = mod[last]
    for j in range(len(rhymes)):
      try:
        vector2= mod[rhymes[j]]
        vector1 = vector1.reshape(1, -1)
        vector2 = vector2.reshape(1, -1)  
        similarity = cosine_similarity(vector1, vector2)
        sim[j]=similarity
      except:
        continue
    sim=pd.DataFrame(sim)
    rhymes=pd.DataFrame(rhymes)
    sim.insert(loc=1,column=1,value=rhymes)
    sim=sim.sort_values(by=0)
    rhymes=list(sim[1])
    line[-1]=rhymes[-3]
    separator = ' '
    lines[i+1]=separator.join(line)
  except:
      continue
sep = "\n"
lines = sep.join(lines)
print(lines)

Fall to the ground and I wake up in your sleep
And it's like watching Casey Anthony walk in your sweep
Thoughts are falling all around you like they were deep
But you're not falling from that tree, you're in flight beep
And I'm standing in your path, your mind is blank
And I'm screaming in your earlobbery like it's my only chance to shut you frank
‘Cause you're not even on your radar hank
And if you think I'm insane, just sit your car on the frank
And I'd rather kill you frank
